In [1]:
import sys
from six.moves import cPickle as pickle
from matplotlib import pyplot
from tensorflow import keras
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Dense
from keras.layers import Flatten
from keras.optimizers import SGD
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dropout
from keras.layers import BatchNormalization

# Load Pickle File

# all_data = pickle.load(open('CIFAR_100_processed.pickle', 'rb'))
# train_data = all_data['train_dataset']
# test_data = all_data['test_dataset']

# train_labels = all_data['train_labels']
# test_labels = all_data['test_labels']
# label_names = all_data['label_names']

# del all_data

# # Print Shape Summary

# print('train_data shape is : %s' % (train_data.shape,))
# print('test_data shape is : %s' % (test_data.shape,))

# test_size = test_data.shape[0]
# train_size = train_data.shape[0]

# # Process Images
# train_data = train_data.astype('float32')
# test_data = test_data.astype('float32')

# train_data = train_data / 255.0
# test_data = test_data / 255.0

# # OHE Labels
# train_labels = to_categorical(train_labels)
# test_labels = to_categorical(test_labels)

train_data shape is : (50000, 32, 32, 3)
test_data shape is : (10000, 32, 32, 3)


In [ ]:
(x_train, y_train), (x_test, y_test) = tensorflow.keras.datasets.cifar100.load_data()

# One-hot encoding for 100 classes.
y_train = to_categorical(y_train, 100)
y_test = to_categorical(y_test, 100)

print('Train: X=%s, y=%s' % (x_train.shape, y_train.shape))
print('Test: X=%s, y=%s' % (x_test.shape, y_test.shape))

In [2]:
def define_model(): 
    model = Sequential()
    model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', input_shape=(32, 32, 3)))
    model.add(BatchNormalization())
    model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D((2, 2)))
    model.add(Dropout(0.2))
    
    model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
    model.add(BatchNormalization())
    model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D((2, 2)))
    model.add(Dropout(0.3))
    
    model.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
    model.add(BatchNormalization())
    model.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D((2, 2)))
    model.add(Dropout(0.4))
    
    model.add(Flatten())
    model.add(Dense(128, activation='relu', kernel_initializer='he_uniform'))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))
    model.add(Dense(100, activation='softmax'))
    
    # compile model
    opt = SGD(lr=0.001, momentum=0.9)
    model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
    return model
 
def summarize_diagnostics(history):
    # plot loss
    pyplot.subplot(211)
    pyplot.title('Cross Entropy Loss')
    pyplot.plot(history.history['loss'], color='blue', label='train')
    pyplot.plot(history.history['val_loss'], color='orange', label='test')
    
    # plot accuracy
    pyplot.subplot(212)
    pyplot.title('Classification Accuracy')
    pyplot.plot(history.history['accuracy'], color='blue', label='train')
    pyplot.plot(history.history['val_accuracy'], color='orange', label='test')
    
    # save plot to file
    filename = sys.argv[0].split('/')[-1]
    pyplot.savefig(filename + '_plot.png')
    pyplot.close()

In [ ]:
# define model
model = define_model()

# fit model
history = model.fit(x_train, y_train, epochs=100, batch_size=64, validation_data=(test_data, test_labels), verbose=1)

# evaluate model
_, acc = model.evaluate(x_test, y_test, verbose=0)
print('> %.3f' % (acc * 100.0))

# learning curves
summarize_diagnostics(history)

Epoch 1/100
782/782 [==============================] - 99s 127ms/step - loss: 4.7864 - accuracy: 0.0383 - val_loss: 3.9877 - val_accuracy: 0.1081
Epoch 2/100
782/782 [==============================] - 101s 129ms/step - loss: 4.1505 - accuracy: 0.0811 - val_loss: 3.8058 - val_accuracy: 0.1375
Epoch 3/100
782/782 [==============================] - 100s 127ms/step - loss: 3.8907 - accuracy: 0.1093 - val_loss: 3.6680 - val_accuracy: 0.1529
Epoch 4/100
782/782 [==============================] - 99s 127ms/step - loss: 3.7133 - accuracy: 0.1333 - val_loss: 3.4980 - val_accuracy: 0.1820
Epoch 5/100
782/782 [==============================] - 99s 126ms/step - loss: 3.5662 - accuracy: 0.1559 - val_loss: 3.3519 - val_accuracy: 0.2074
Epoch 6/100
782/782 [==============================] - 101s 130ms/step - loss: 3.4427 - accuracy: 0.1762 - val_loss: 3.3512 - val_accuracy: 0.2044
Epoch 7/100
782/782 [==============================] - 101s 129ms/step - loss: 3.3224 - accuracy: 0.1987 - val_loss: 3.25